In [ ]:

#######################################################
#######################################################
############    COPYRIGHT - DATA SOCIETY   ############
#######################################################
#######################################################

## 10 TIMESERIES DAY4 ##

## NOTE: To run individual pieces of code, select the line of code and
##       press ctrl + enter for PCs or command + enter for Macs



In [ ]:
#=================================================-
#### Slide 9: Import packages  ####

import os
import pandas as pd
from pandas.plotting import lag_plot
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt



In [ ]:
#=================================================-
#### Slide 10: Directory settings  ####

# Set `main_dir` to the location of your `af-werx` folder (for Linux).
main_dir = "/home/[username]/Desktop/af-werx"
# Set `main_dir` to the location of your `af-werx` folder (for Mac).
main_dir = '/Users/[username]/Desktop/af-werx'
# Set `main_dir` to the location of your `af-werx` folder (for Windows).
main_dir = "C:\\Users\\[username]\\Desktop\\af-werx"
# Make `data_dir` from the `main_dir` and
# remainder of the path to data directory.
data_dir = main_dir + "/data"




In [ ]:
#=================================================-
#### Slide 11: Working directory  ####

# Set working directory.
os.chdir(data_dir)
# Check working directory.
print(os.getcwd())



In [ ]:
#=================================================-
#### Slide 12: Load passenger miles dataset  ####

# Read pickle file into `passenger_miles` variable.
passenger_miles = pickle.load(open((data_dir + "/passenger_miles.sav"),"rb"))
print(passenger_miles.head())




In [ ]:
#=================================================-
#### Slide 13: Recap: stationarity test  ####

# Perform ADF test on original series.
result_pm = adfuller(passenger_miles['revenue_passenger_miles'])

print('ADF Statistic: %f' % result_pm[0])
print('p-value: %f' % result_pm[1])
print('Critical Values:')
for key, value in result_pm[4].items():
    print('\t%s: %.3f' % (key, value))



In [ ]:
#=================================================-
#### Slide 14: Test deseasonalized series for stationarity  ####

# Decompose revenue passenger miles into its components.
res = seasonal_decompose(passenger_miles['revenue_passenger_miles'])

# Deseasonalized series (Y_t - S_t = T_t + e_t).
deseasonalized = passenger_miles['revenue_passenger_miles'] - res.seasonal




In [ ]:
#=================================================-
#### Slide 15: Test deseasonalized series for stationarity  ####

# Perform ADF test on Y_t - T_t - S_t series.
result_pm_deseason = adfuller(deseasonalized)

print('ADF Statistic: %f' % result_pm_deseason[0])
print('p-value: %f' % result_pm_deseason[1])
print('Critical Values:')
for key, value in result_pm_deseason[4].items():
    print('\t%s: %.3f' % (key, value))



In [ ]:
#=================================================-
#### Slide 16: Test decomposed series for stationarity  ####

print(res.resid.head(10))



In [ ]:
#=================================================-
#### Slide 17: Test decomposed series for stationarity  ####

# Remove NaN values
# (the test won't work otherwise).
residuals = res.resid[~ np.isnan(res.resid)]
print(residuals.head(10))
# Perform ADF test on Y_t - T_t - S_t series.
result_pm_resid = adfuller(residuals)

print('ADF Statistic: %f' % result_pm_resid[0])
print('p-value: %f' % result_pm_resid[1])
print('Critical Values:')
for key, value in result_pm_resid[4].items():
    print('\t%s: %.3f' % (key, value))



In [ ]:
#=================================================-
#### Slide 23: Exercise 1  ####





In [ ]:
#=================================================-
#### Slide 59: Plot ACF vs PACF  ####

# Take difference.
diff = list()
for i in range(1, len(deseasonalized)):
    value = deseasonalized[i] - deseasonalized[i - 1]
    diff.append(value)

# Convert to series with DatetimeIndex.
deseasonalized_diff = pd.Series(diff, index = deseasonalized.index[1:])
print(deseasonalized_diff.head(3))



In [ ]:
#=================================================-
#### Slide 64: Exercise 2  ####





In [ ]:
#=================================================-
#### Slide 67: Constructing ARIMA model: set up model  ####

# Fit ARIMA model.
arima = ARIMA(deseasonalized,
order = (12, 1, 0)) #<- add p, d, q components here
print(arima)



In [ ]:
#=================================================-
#### Slide 68: Constructing ARIMA model: fit model  ####

# Fit model and set `disp` to 0 to avoid
# model debugging info print in console.
arima_fit = arima.fit(disp = 0)
print(arima_fit.summary().tables[0])



In [ ]:
#=================================================-
#### Slide 69: Constructing ARIMA model: results (cont'd)  ####

print(arima_fit.summary().tables[1])



In [ ]:
#=================================================-
#### Slide 70: Constructing ARIMA model: results (cont'd)  ####

print(arima_fit.summary().tables[2])



In [ ]:
#=================================================-
#### Slide 71: Constructing ARIMA model: check residuals  ####

# Plot residuals.
residuals = pd.DataFrame(arima_fit.resid)
print(residuals.describe())
fig, axes = plt.subplots(nrows = 2,
ncols = 1,
figsize = (7,7))
residuals.plot(ax = axes[0])
residuals.plot(ax = axes[1], kind='kde')
plt.show()



In [ ]:
#=================================================-
#### Slide 72: Constructing ARIMA model: residuals (cont'd)  ####

fig, axes = plt.subplots(nrows = 2,
ncols = 1,
figsize = (8,8))
plot_acf(residuals, ax = axes[0])
plot_pacf(residuals, ax = axes[1])
plt.show()



In [ ]:
#=================================================-
#### Slide 73: Forecasting using ARIMA model: prep data  ####

# Set a cutoff point for train-test.
cutoff = int(len(deseasonalized) * 0.90)
# Split data into train and test.
train, test = deseasonalized[0:cutoff], deseasonalized[cutoff:len(deseasonalized)]




In [ ]:
#=================================================-
#### Slide 75: Forecasting using ARIMA model (cont'd)  ####

# Set up ARIMA model using train data only.
arima = ARIMA(train, order = (12, 1, 0))
# Fit ARIMA model to train data only.
arima_fit = arima.fit(disp = 0)
# Generate ARIMA model forecast.
output = arima_fit.forecast(steps = len(test))
# Get predictions from forecast output.
predictions = output[0]




In [ ]:
#=================================================-
#### Slide 76: Forecasting using ARIMA model (cont'd)  ####

# Compare the raw `DatetimeIndex`.
print(test.index[0])

# With formatted `DatetimeIndex`.
print(test.index[0].strftime("%Y-%m"))




In [ ]:
#=================================================-
#### Slide 77: Forecasting using ARIMA model (cont'd)  ####

# Show mean squared error.
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)

# Show RMSE (i.e. metric in the same units as our data).
print('RMSE: %.3f' % sqrt(error))



In [ ]:
#=================================================-
#### Slide 78: View results: actual vs forecast  ####

# Convert predictions list into series with DatetimeIndex.
revenue_miles_forecast = pd.Series(predictions, index = test.index)
# Plot actual vs forecast values.
fig, ax = plt.subplots(figsize = (15, 6))
ax.plot(test,
marker = 'o',
linestyle = '-',
linewidth = 1.5,
label = 'Actual value')
ax.plot(revenue_miles_forecast,
marker = 'o',
markersize = 5,
linewidth = 2.5,
linestyle = '-',
label = 'Forecast value')
ax.set_xlabel('Date', fontsize = 18)
ax.set_title("Actual vs forecast values for ARIMA(12, 1, 0)", fontsize = 20)
ax.legend()
plt.show()



In [ ]:
#=================================================-
#### Slide 81: View results: the big picture  ####

plt.close(fig="all") #<- closes all previous plots and figures
# Plot actual vs forecast values.
fig, ax = plt.subplots(figsize = (15, 6))
ax.plot(train,
marker = 'o',
linestyle = '-',
linewidth = 1.5,
label = 'Train')
arima_fit.plot_predict('2000-01', '2002-04', #<- date range for forecast values
ax = ax,
plot_insample = False)#<- forecast is done for out of sample values
ax.plot(test,
marker = '.',
linestyle = '-',
label = 'Test (actual value)')
ax.set_xlabel('Date', fontsize = 18)
ax.set_title("Actual vs forecast values for ARIMA(12, 1, 0)", fontsize = 20)
ax.legend()
plt.show()

